In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler, maxabs_scale, MaxAbsScaler
import time
import random


In [16]:
def balance_data(df):

    for i in range(1, 8):
        print(i)
        cond = df.Cover_Type == i
        subset = df[cond].dropna()#removes the rows that contains NULL values
        if i == 4:
            subset = subset.sample(n=1000) # achieve this in pandas by using the "sample" method with the n parameter
        else:
            subset = subset.sample(n=1000)
            
        if i == 1:
            balanced_data = subset.sample(n=1000)
        else:
            balanced_data = balanced_data._append(subset)

    return balanced_data

In [6]:
def getData(filename):
    chunksize = 1200000
    flag =1
    for data in pd.read_csv(filename, sep=",", chunksize=chunksize):
        while flag<2 and chunksize < 1200000:
            print(data)
        flag+=1
    return data

In [8]:
filename = '../Dataset/covtype.csv'
dataset=getData(filename)

In [9]:
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


# get short data

In [42]:
df=balance_data(dataset)

1
2
3
4
5
6
7


In [43]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
411904,3203.0,34.0,13.0,391.0,131.0,1024.0,218.0,212.0,128.0,616.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
224899,3381.0,331.0,11.0,360.0,41.0,3242.0,194.0,226.0,172.0,1781.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
15963,2681.0,5.0,29.0,150.0,75.0,2251.0,168.0,169.0,126.0,7050.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
419881,3303.0,283.0,11.0,134.0,22.0,1637.0,191.0,241.0,190.0,1544.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14322,2930.0,18.0,22.0,120.0,71.0,1136.0,197.0,188.0,121.0,3291.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Normalized

In [44]:
def normalize_data(rawData):
    # training
    norm_tcolumns = rawData[rawData.columns[:10]]  # only the first ten columns need normalization, the rest is binary
    #     scaler = MinMaxScaler(copy=True, feature_range=(0, 1)).fit(norm_tcolumns.values)
    scaler = MaxAbsScaler(copy=True).fit(norm_tcolumns.values)
    scaledf = scaler.transform(norm_tcolumns.values)
    
    training_examples = pd.DataFrame(scaledf, index=norm_tcolumns.index,
                                     columns=norm_tcolumns.columns)  # scaledf is converted from array to dataframe
    rawData.update(training_examples)

    return rawData

In [45]:
ExperimentDataset=normalize_data(df)

In [46]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
411904,0.833030,0.094444,0.270833,0.302164,0.325871,0.156145,0.858268,0.834646,0.516129,0.087351,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
224899,0.879324,0.919444,0.229167,0.278207,0.101990,0.494358,0.763780,0.889764,0.693548,0.252552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
15963,0.697269,0.013889,0.604167,0.115920,0.186567,0.343245,0.661417,0.665354,0.508065,0.999716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
419881,0.859038,0.786111,0.229167,0.103555,0.054726,0.249619,0.751969,0.948819,0.766129,0.218945,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14322,0.762029,0.050000,0.458333,0.092736,0.176617,0.173224,0.775591,0.740157,0.487903,0.466676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479023,0.861118,0.872222,0.541667,0.315301,0.320896,0.665599,0.562992,0.822835,0.814516,0.347703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0
14357,0.870481,0.363889,0.104167,0.131376,0.062189,0.277219,0.901575,0.933071,0.572581,0.454765,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0
378069,0.887386,0.969444,0.208333,0.477589,0.181592,0.223544,0.795276,0.881890,0.645161,0.211288,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0
292752,0.878283,0.825000,0.166667,0.073416,0.022388,0.643794,0.779528,0.937008,0.721774,0.593307,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0


# SAVE as csv

In [31]:
df_column=df.columns
df_column

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [50]:
import random
df_list=df.values.tolist()
random.shuffle(df_list)
dataset=pd.DataFrame(df_list,columns=df_column)

In [51]:
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,0.667100,0.338889,0.312500,0.051777,0.049751,0.268527,0.960630,0.893701,0.431452,0.251276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,0.561508,0.741667,0.520833,0.083462,0.114428,0.123513,0.578740,0.944882,0.915323,0.042541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,0.742263,0.283333,0.145833,0.348532,0.201493,0.312900,0.913386,0.909449,0.532258,0.411656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.604681,0.344444,0.562500,0.187017,0.318408,0.121836,0.996063,0.822835,0.262097,0.215258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,0.672822,0.316667,0.229167,0.176198,-0.017413,0.280878,0.944882,0.901575,0.475806,0.280913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [52]:
dataset.to_csv('DatasetForExperiment.csv', index=False)